In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import nest_asyncio

nest_asyncio.apply()



In [3]:
import logging
import os
from datetime import datetime
from pathlib import Path

from dotenv import load_dotenv

from econagents.core.game_runner import GameRunner, HybridGameRunnerConfig
from examples.ibex_tudelft.voting.manager import HLAgentManager
from examples.ibex_tudelft.create_game import create_game_from_specs

load_dotenv()

# Load environment variables
HOSTNAME = os.getenv("HOSTNAME")
PORT = os.getenv("PORT")
USERNAME = os.getenv("GAME_USERNAME")
PASSWORD = os.getenv("GAME_PASSWORD")

In [4]:
# Create a new game
new_game_data = create_game_from_specs(
    specs_path=Path().absolute() / "voting.json",
    base_url=f"http://{HOSTNAME}",
    game_name=f"voting {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
    credentials={"username": USERNAME, "password": PASSWORD},
)
game_id = new_game_data["game_id"]
num_agents = new_game_data["num_agents"]
login_payloads = new_game_data["login_payloads"]


# Run the game
config = HybridGameRunnerConfig(
    # Game configuration
    game_id=game_id,
    # Server configuration
    protocol="ws",
    hostname=HOSTNAME,
    port=int(PORT),
    path="wss",
    # Continuous-time phase configuration
    continuous_phases=[2,5],
    min_action_delay=2,
    max_action_delay=7,
    # Logging configuration
    log_level=logging.DEBUG,
)
agents = [HLAgentManager(game_id=game_id, auth_mechanism_kwargs=login_payloads[i]) for i in range(num_agents)]
runner = GameRunner(agents=agents, config=config)
await runner.run_game()


INFO:examples.ibex_tudelft.create_game:Total number of agents: 6
INFO:examples.ibex_tudelft.create_game:Creating new game...
INFO:examples.ibex_tudelft.create_game:Creating game with payload: {'username': 'dylan', 'password': 'password': 'REDACTED', 'gameParameters': {'title': 'voting 2025-03-27 14:27:40', 'tax_rate': {'initial': 1, 'final': 33}, 'signal': {'low': 0.95, 'high': 1.05, 'generate': False}, 'speculators': {'count': 0, 'balance': 50000, 'shares': 5, 'max_lot_purchases': 3, 'cash_for_snipers': 250000, 'base_points': 400000, 'reward_scale_factor': 20000}, 'developers': {'count': 1, 'balance': 300000, 'shares': 30, 'base_points': 0, 'reward_scale_factor': 50000, 'profit': {'no_project': {'low': 200000, 'fixed': 350000, 'high': 500000}, 'project_a': {'low': 500000, 'fixed': 1150000, 'high': 2750000}, 'project_b': {'low': 1500000, 'fixed': 1750000, 'high': 2000000}}}, 'owners': {'count': 5, 'balance': 60000, 'shares': 6, 'base_points': 200000, 'reward_scale_factor': 20000, 'prof

CancelledError: 